In [1]:
import numpy as np 
import pandas as pd
import scipy
from scipy.spatial.distance import cosine
from pandas import DataFrame

from tkinter import *
from tkinter.scrolledtext import ScrolledText
from tkinter import font

import matplotlib
import matplotlib.pyplot as plt

from ipynb.fs.full.Data_preprocessing import clean

import matplotlib.pyplot as plt
from pandas import DataFrame

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import pickle
import gensim

from ipynb.fs.full.Data_preprocessing import clean

import xgboost as xgb

'''
To import any module x
import sys
!{sys.executable} -m pip install x

'''



'\nTo import any module x\nimport sys\n!{sys.executable} -m pip install x\n\n'

In [2]:
df = pd.read_csv('AnsData.csv', header = None, sep='\t' )
data = pd.read_csv("quora_duplicate_questions.tsv", sep="\t")



In [3]:

from ipynb.fs.full.Data_preprocessing import clean

data['question1'] = data['question1'].apply(clean)
data['question2'] = data['question2'].apply(clean)

In [4]:

model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
model.init_sims(replace=True)



In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_features = 50000)
tfidf_vec.fit(pd.Series(data['question1']+data['question2']) .astype(str).unique() ) 

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [6]:

from sklearn.feature_extraction.text import TfidfVectorizer
ntfidf_vect = TfidfVectorizer(max_features = 50000, ngram_range=(2,2))
ntfidf_vect.fit(pd.Series(data['question1']+data['question2']) .astype(str).unique() ) 

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(2, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [27]:
def predict(q1, q2, ans1, ans2):
    
    info=""
    q1 = clean(q1)
    q2 = clean(q2)
    ans1 = clean(ans1)
    ans2 = clean(ans2)
    
    da = distance(ans1, ans2)
    dq = distance(q1, q2)

    
    t1 = tfidf_vec.transform([ans1])
    t2 = tfidf_vec.transform([ans2])
    tda = cosine_similarity(t1, t2)

    
    t1 = tfidf_vec.transform([q1])
    t2 = tfidf_vec.transform([q2])
    tdq = cosine_similarity(t1, t2)

    X = np.zeros((1,4))
    
    X[0][0] = dq
    X[0][1] = da
    X[0][2] = tdq
    X[0][3] = tda

    print(X)
    
    print(logreg.predict(X))
    print(random_forest.predict(X))
    print(linear_svc.predict(X))
    print(sgd.predict(X))
    print(gaussian.predict(X))
    print(knn.predict(X))
    print(votingC.predict(X))
    
    info+= "Is Duplicate:\n"
    info+="Using Logistic Regression: {}\n".format(logreg .predict(X))
    info+="Using Random Forest: {}\n".format(random_forest.predict(X))
    info+="Using Linear SVC: {}\n".format( linear_svc.predict(X))
    info+="Using SGD: {}\n".format( sgd.predict(X))
    info+="Using Guassian NB: {}\n".format(gaussian.predict(X))
    info+="Using KNN: {}\n".format( knn.predict(X))
    info+="Using VotingC: {}\n".format(votingC.predict(X))
    
    return info
    

In [8]:

def cosine_similarity(tfidf_1, tfidf_2):
    return cosine(tfidf_1.todense(),tfidf_2.todense())


def wmd(s1,s2):
    return model.wmdistance(s1, s2)

def distance(s1, s2):
    
    
    num = len(s1)
          
    sent1 = [word for word in s1.split() if word in model.vocab]
    sent2 = [word for word in s2.split() if word in model.vocab]
        
    if len(sent1)>0 and len(sent2)>0:
        return model.wmdistance(sent1,sent2)
    else:
        return 1000

In [9]:
dist = pd.read_csv('wmd_distance_ans.csv', header=None, index_col=None)

In [12]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

random_forest = RandomForestClassifier(n_estimators=5)
random_forest.fit(X_train, Y_train)

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)

votingC = VotingClassifier(estimators=[('lg', logreg), ('rf', random_forest),
 ('svc' , linear_svc),('sgd',sgd),('gbc',gaussian), ('knn', knn)])
votingC = votingC.fit(X_train, Y_train)

/home/stud/btech/cse/2017/rahul.cs17/anaconda3/envs/rahul/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/stud/btech/cse/2017/rahul.cs17/anaconda3/envs/rahul/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/stud/btech/cse/2017/rahul.cs17/anaconda3/envs/rahul/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/stud/btech/cse/2017/rahul.cs17/anaconda3/envs/rahul/lib/python3.6/

In [10]:

train_data, test_data = train_test_split(dist, test_size = 0.1, random_state = 0)
X_train, Y_train = train_data.loc[:, :3], train_data.loc[:, 4]
X_test, Y_test = test_data.loc[:, :3], test_data.loc[:, 4]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [26]:


root = Tk()
root.title('Find similarity between two questions')


def callback():
    
    # Get both questions
    q1= que1.get(1.0, END)
    a1= ans1.get(1.0, END)
    q2= que2.get(1.0, END)
    a2= ans2.get(1.0, END)
    
    # Open new window
    new_window = Toplevel(root)
    new_window.title('Similarity between two questions')
    
    
    
    # Printing question 1
    text1 = Message(new_window, text= 'Question1:\n\n' + q1, anchor = 'w',  aspect = 1000)
    text1.pack(fill = X)
    
    # Printing question 2
    text2 = Message(new_window, text= '\n\nAnswer1:\n\n' + a1 , anchor = 'w',  aspect = 1000)
    text2.pack(fill = X)
    
    # Printing ans 1
    text1 = Message(new_window, text= 'Question2:\n\n' + q2, anchor = 'w',  aspect = 1000)
    text1.pack(fill = X)
    
    # Printing ans 2
    text2 = Message(new_window, text= '\n\nAnswer2:\n\n' + a2 , anchor = 'w',  aspect = 1000)
    text2.pack(fill = X)
    
    # Cleaning both questions
    q1 = clean(q1)
    q2 = clean(q2)
    a1 = clean(a1)
    a2 = clean(a2)
    
    
    # Using word2vec model
    sent1 = [word for word in q1.split() if word in model.vocab]
    sent2 = [word for word in q2.split() if word in model.vocab]
    
    l1 = Label( new_window, text= '\n\nUsing word2vec:\n\n', anchor = 'w' )
    l1.pack(fill = X)
    text3 = Message(new_window, text='Distance:\n\n'+ str(model.wmdistance(sent1, sent2)), anchor = 'w', aspect = 500)
    text3.pack(fill = X)
    
    
    # Using unigram tfidf with cosine similarity
    uni_dist = str( cosine_similarity( tfidf_vec.transform([q1]) , tfidf_vec.transform([q2]) ) )
   
    
    l2 = Label( new_window, text='\n\nUsing unigram:\n\n',  anchor = 'w')
    l2.pack(fill = X)  
    text4 = Message(new_window, text='Distance\n\n: '+ uni_dist, anchor = 'w',  aspect = 1000)
    text4.pack(fill = X)
    
    
    
    # Using ngram tfidf with cosine similarity
    n_dist = str( cosine_similarity( ntfidf_vect.transform([q1]) , ntfidf_vect.transform([q2]) ) )
        
    l3 = Label( new_window, text='\n\nUsing n-gram:\n\n',  anchor = 'w')
    l3.pack(fill = X)   
    text5 = Message(new_window, text='Distance\n\n: '+ n_dist, anchor = 'w', aspect = 1000)
    text5.pack(fill = X)
    
    l4 = Label( new_window, text='\n\nUsing answers:\n\n',  anchor = 'w')
    l4.pack(fill = X)   
    
    ans =""
    if len(a1)==0 or len(a2)==0:
        ans ="Answer NOT FOUND"
    else:
        ans = predict(q1, q2, a1, a2)
    print(ans)    
    text5 = Message(new_window, text=ans, anchor = 'w',  aspect = 1000)
    text5.pack(fill = X)
    
    
    
# Fonts used 
Font1 = font.Font(family="Comic Sans MS", size=10)
Font2 = font.Font(family="Arial", size=9)
   

# top Frame to contain 1st question
topFrameq = Frame(root, padx = 10, pady=5)
topFrameq.pack()

# top Frame to contain 1st ans
topFramea = Frame(root, padx = 10, pady=5)
topFramea.pack()

# Mid Frame to contain 2nd question
MidFrameq = Frame(root, padx=10, pady=5)
MidFrameq.pack()

# Mid Frame to contain 2nd question
MidFramea = Frame(root, padx=10, pady=5)
MidFramea.pack()

# Bottom frame to contain button
bottomFrame = Frame(root, padx = 5, pady=5)
bottomFrame.pack(side= BOTTOM)

# Top Frame
label1 = Label( topFrameq, text='Enter Question1:', font=Font1)
label1.pack(side=LEFT)
que1 = Text(topFrameq, padx = 10, pady =10, font=Font2, height=7)
que1.focus_set()
que1.pack(side = LEFT)

label3 = Label( topFramea, text='Enter Answer 1:', font=Font1)
label3.pack(side=LEFT)
ans1 = Text(topFramea, padx = 10, pady =10, font=Font2, height=13)
ans1.focus_set()
ans1.pack(side = LEFT)

# Mid Frame
label2 = Label(MidFrameq , text='Enter Question2:', font=Font1)
label2.pack(side=LEFT)
que2 = Text(MidFrameq,  padx = 10, pady =10, font=Font2, height=7)
que2.focus_set()
que2.pack(side = LEFT)

label3 = Label(MidFramea , text='Enter Answer 2:', font=Font1)
label3.pack(side=LEFT)
ans2 = Text(MidFramea, padx = 15, pady =10, font=Font2, height=13)
ans2.focus_set()
ans2.pack(side = LEFT)
 
# Bottom Frame
b = Button(bottomFrame, text="Find similarity", width=20, fg='black', activebackground='green' ,command=callback)
b.pack()

root.mainloop()

[[0.16682849 1.31598484 0.3328098  1.        ]]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
Is Duplicate:
Using Logistic Regression: [0.]Using Random Forest: [0.]Using Linear SVC: [0.]Using SGD: [0.]Using Guassian NB: [0.]Using KNN: [1.]Using VotingC: [0.]


In [ ]:
# import pickle

# # Word2vec model
# f = open('Google_word2vec.pickle', 'rb')
# model = pickle.load(f)
# f.close()

# # ntfidf with cosine similarity
# f = open('ntfidf_vec.pickle', 'rb')
# ntfidf_vec = pickle.load(f)
# f.close()

# # tfidf with cosine similarity
# f = open('tfidf_vec.pickle', 'rb')
# tfidf_vec = pickle.load(f)
# f.close()

# # tfidf with xgboost

# f = open('tfidf_vec_xgb.pickle', 'rb')
# tfidf_vec_xgb = pickle.load(f)
# f.close()

# f = open('tfidf_xgb.pickle', 'rb')
# tfidf_xgb = pickle.load(f)
# f.close()

# # ntfidf with xgboost

# f = open('ntfidf_vec_xgb.pickle', 'rb')
# ntfidf_vec_xgb = pickle.load(f)
# f.close()


# f = open('ntfidf_xgb.pickle', 'rb')
# ntfidf_xgb = pickle.load(f)
# f.close()


# # f = open('bst_xgb.pickle', 'rb')
# # bst = pickle.load(f)
# # f.close()
